# Example VCF and Annotation for the test_all v01 Gene Panel

This is just to demonstrate how you could hammer on public databases to demonstrate some dummy VCFs for one or more RS#s in order to test out Ella functionality.

This assumes you created the gene panel from `Understanding-The-Ella-Datamodel/GenePanels/Create-Custom-GenePanels-From-Public-Databases.ipynb`.

In [1]:
import os
import pandas as pd
import requests
from pprint import pprint
import numpy as np
from datetime import datetime
import json

Keep in mind that each analysis must be associated to a gene panel. An analysis name could be a sample name, a family name, population, etc.

In this case we are saying we have a sample `test_sample_1`, that we are using as an analysis name, with a gene panel `test_genepanel` version `v01`.

In [2]:
SAMPLE_ID="test_sample_1"
GENEPANEL_NAME="test_genepanel"
GENEPANEL_VERSION="v01"

ANALYSIS_NAME="{SAMPLE_ID}.{GENEPANEL_NAME}_{GENEPANEL_VERSION}".format(SAMPLE_ID=SAMPLE_ID, 
                                                                        GENEPANEL_NAME=GENEPANEL_NAME, 
                                                                        GENEPANEL_VERSION=GENEPANEL_VERSION,
                                                                       )
BASE_PATH="/data"
ANALYSIS_DIR=os.path.join(BASE_PATH,  "analysis", "incoming", ANALYSIS_NAME)
GENEPANEL_DIR=os.path.join(BASE_PATH,  "genepanels")
os.makedirs(ANALYSIS_DIR, exist_ok=True)
os.makedirs(GENEPANEL_DIR, exist_ok=True)

## Prepare the VCF

VCFs have one or more Info lines, along with the standard VCF columns.

In [3]:
vcf_info_line = """##fileformat=VCFv4.1
##contig=<ID=13>
##FILTER=<ID=PASS,Description="All filters passed">
"""

vcf_columns = ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT"]
vcf_columns.append(SAMPLE_ID)

def get_genepanel_info_line(genepanel_name, genepanel_version, date=None):
    if date is None:
        date = datetime.today().strftime('%Y-%m-%d')
    return "# Genepanel: {} Version: {} Date: {}\n".format(genepanel_name, genepanel_version, date)


def write_pandas_csv_with_info_line(file_name, info_line, df):
    with open(file_name, 'w') as fp:
        fp.write(info_line)
        df.to_csv(fp, index=False, sep="\t")

## Get Variant Info Data from MyVariant

Once we have the RS number we can get the rest of the information from MyVariant.info

In [4]:
def get_variant_data_myvariant(query):
    variant_data = requests.get("https://myvariant.info/v1/variant/{}".format(query))
    variant_data = variant_data.json()
    #assert '_id' in variant_data
    return {
        #"hgvs": variant_data["_id"],
        "chr": variant_data["chrom"],
        "rsid": variant_data["dbsnp"]["rsid"],
        "ref": variant_data["dbsnp"]["ref"],
        "alt": variant_data["dbsnp"]["alt"],
        "start": variant_data["hg19"]["start"],
        "end": variant_data["hg19"]["end"],
    }


## Build VCFs

In [5]:
example_row = [13, 32972626, "CM082515",  "A", "T",  5000.0,  "PASS",    ".",  "GT:AD:DP:GQ:PL",  "0/1:107,80:187:99:2048,0,2917"]
# I'm not sure how the ids in the VCF file are generated so i'm just going to add one for each for
first_id = 82515

## BRCA VCF

BRCA2 NM_000059.4 c.9976A>T p.Lys3326Ter

[GnomAD p.Lys3326Ter](https://gnomad.broadinstitute.org/variant/13-32972626-A-T?dataset=gnomad_r2_1) 

[DBSnp rs11571833](https://www.ncbi.nlm.nih.gov/projects/SNP/snp_ref.cgi?rs=rs11571833)

**Genome Coordinates:** GRCh37.p13 chr 13	NC_000013.10:g.32972626A>T

chr 13 32972626 A>T

In [6]:
snp_record = get_variant_data_myvariant("rs11571833")
pd.DataFrame.from_records([snp_record])

,chr,rsid,ref,alt,start,end
0,13,rs11571833,A,T,32972626,32972626


In [7]:
brca2_vcf_record = {'#CHROM': snp_record['chr'],
 'POS': snp_record['start'],
 'ID': 'CM082515',
 'REF': snp_record['ref'],
 'ALT': snp_record['alt'],
 'QUAL': 5000.0,
 'FILTER': 'PASS',
 'INFO': '.',
 'FORMAT': 'GT:AD:DP:GQ:PL',
 'test_sample_1': '0/1:107,80:187:99:2048,0,2917'}

## HBB c.33C>A p.Ala11Ala

HBB	NM_000518.5	c.33C>A	p.Ala11Ala

https://www.ncbi.nlm.nih.gov/gene/3043

Ensembl:ENSG00000244734 MIM:141900

SNP Info for p.Ala11Ala

RS - https://www.ncbi.nlm.nih.gov/snp/rs35799536

GRCh37.p13 chr 11	NC_000011.9:g.5248219G>A

GRCh37.p13 chr 11	NC_000011.9:g.5248219G>T

I can't find the exact phenotype the variant scientist is looking for in Gnomad.  I can find `p.Ala11Ala`, but not `c.33C>A`

### Clinvar c.33C>A	p.Ala11Ala

[Clinvar](https://www.ncbi.nlm.nih.gov/clinvar/variation/439155/)

11: 5248219 (GRCh37) C -> A

In [8]:
# TODO This one comes up as not a unique record
# So I am just manually adding in the record
#snp_record = get_variant_data_myvariant("rs35799536")
#pd.DataFrame.from_records([snp_record])

In [9]:
#variant_data = requests.get("https://myvariant.info/v1/variant/{}".format("rs35799536"))
#variant_data = variant_data.json()
#variant_data

In [10]:
hbb_1_vcf_record = {'#CHROM': 11,
 'POS': 5248219,
 'ID': 'CM0{}'.format(str(first_id + 1)),
 'REF': 'G',
 'ALT': 'A',
 'QUAL': 5000.0,
 'FILTER': 'PASS',
 'INFO': '.',
 'FORMAT': 'GT:AD:DP:GQ:PL',
 'test_sample_1': '0/1:107,80:187:99:2048,0,2917'}
                      

## HBB - c.316-197C>T NA

[Clinvar](https://www.ncbi.nlm.nih.gov/clinvar/RCV000029984/)

[OMIM](http://www.omim.org/entry/141900)

[DBSNP](https://www.ncbi.nlm.nih.gov/snp/rs34451549)

**Location:** Chr11: 5247153 (on Assembly GRCh37)

**Prefered Name:** NM_000518.5(HBB):c.316-197C>T

GRCh37.p13 chr 11	NC_000011.9:g.5247153G>A

In [11]:
hbb_2_vcf_record = {'#CHROM': 11,
 'POS': 5247153,
 'ID': 'CM0{}'.format(str(first_id + 2)),
 'REF': 'G',
 'ALT': 'A',
 'QUAL': 5000.0,
 'FILTER': 'PASS',
 'INFO': '.',
 'FORMAT': 'GT:AD:DP:GQ:PL',
 'test_sample_1': '0/1:107,80:187:99:2048,0,2917'}

## MSH6 - NM_000179.2 c.30C>A  p.Phe10Leu

MSH6	NM_000179.2	c.30C>A 	p.Phe10Leu

eGeneId ENSG00000116062.10

Gnomad Variant ID for p.Phe10Leu - https://gnomad.broadinstitute.org/variant/2-48010400-T-C?dataset=gnomad_r2_1

RS - https://www.ncbi.nlm.nih.gov/snp/rs773861137

GRCh37.p13 chr 2	NC_000002.11:g.48010400T>C

In [12]:
msh6_vcf_record = {'#CHROM': 2,
 'POS': 480104003,
 'ID': 'CM0{}'.format(str(first_id + 3)),
# the vcf annotator is telling me that their shouldn't be a reference allele here
 'REF': '',
 'ALT': 'C',
 'QUAL': 5000.0,
 'FILTER': 'PASS',
 'INFO': '.',
 'FORMAT': 'GT:AD:DP:GQ:PL',
 'test_sample_1': '0/1:107,80:187:99:2048,0,2917'}

## RET	NM_020975.6	c.1832G>A	 p.Cys611Tyr

Clinvar - p.Cys611Tyr
NM_020975.6(RET):c.1832G>A (p.Cys611Tyr)

[Clinvar](https://www.ncbi.nlm.nih.gov/clinvar/RCV000412987/)

Chr10: 43609076 (on Assembly GRCh37)

In [13]:
ret_vcf_record = {'#CHROM': 10,
 'POS': 43609076,
 'ID': 'CM0{}'.format(str(first_id + 4)),
 'REF': 'G',
 'ALT': 'A',
 'QUAL': 5000.0,
 'FILTER': 'PASS',
 'INFO': '.',
 'FORMAT': 'GT:AD:DP:GQ:PL',
 'test_sample_1': '0/1:107,80:187:99:2048,0,2917'}

In [14]:
file_name = os.path.join(ANALYSIS_DIR, "test.vcf")

vcf_records = [brca2_vcf_record, hbb_2_vcf_record ,hbb_1_vcf_record, msh6_vcf_record, ret_vcf_record]
df = pd.DataFrame(columns = vcf_columns, data=vcf_records)

with open(file_name, 'w') as fp:
    fp.write(vcf_info_line)
    df.to_csv(fp, index=False, sep="\t")
    
df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,test_sample_1
0,13,32972626,CM082515,A,T,5000.0,PASS,.,GT:AD:DP:GQ:PL,"0/1:107,80:187:99:2048,0,2917"
1,11,5247153,CM082517,G,A,5000.0,PASS,.,GT:AD:DP:GQ:PL,"0/1:107,80:187:99:2048,0,2917"
2,11,5248219,CM082516,G,A,5000.0,PASS,.,GT:AD:DP:GQ:PL,"0/1:107,80:187:99:2048,0,2917"
3,2,480104003,CM082518,,C,5000.0,PASS,.,GT:AD:DP:GQ:PL,"0/1:107,80:187:99:2048,0,2917"
4,10,43609076,CM082519,G,A,5000.0,PASS,.,GT:AD:DP:GQ:PL,"0/1:107,80:187:99:2048,0,2917"
